In [63]:
import torch
import dgl
import numpy as np
from rfdiffusion.SE3_network import SE3TransformerWrapper
from rfdiffusion.util_module import make_topk_graph, rbf, get_seqsep

def create_batch_with_padding(real_lengths=None):
    """
    创建包含padding的batch数据，模拟不等长序列
    """
    batch_size = 3
    max_seq_len = 100
    if real_lengths is None:
        real_lengths = [75, 60, 85]  # 真实序列长度
    else:
        real_lengths = [min(length, max_seq_len) for length in real_lengths]
    
    print(f"Creating batch data with padding...")
    print(f"Batch size: {batch_size}")
    print(f"Max sequence length: {max_seq_len}")
    print(f"Real sequence lengths: {real_lengths}")
    
    # 初始化数据
    xyz = torch.randn(batch_size, max_seq_len, 3, 3)  # CA坐标 (B, L, 3, 3)
    pair = torch.randn(batch_size, max_seq_len, max_seq_len, 128)  # pair特征
    idx = torch.arange(max_seq_len).unsqueeze(0).repeat(batch_size, 1)  # 残基索引
    
    # 创建mask并添加NaN padding
    mask = torch.zeros(batch_size, max_seq_len, dtype=torch.bool)
    
    for i, length in enumerate(real_lengths):
        # 标记有效位置
        mask[i, :length] = True
        
        # 在padding位置添加NaN
        xyz[i, length:] = float('nan')
        pair[i, length:] = float('nan')
        pair[i, :, length:] = float('nan')
    
    print(f"Data shapes:")
    print(f"  XYZ coordinates: {xyz.shape}")
    print(f"  Pair features: {pair.shape}")
    print(f"  Residue indices: {idx.shape}")
    print(f"  Valid mask: {mask.shape}")
    print(f"  Contains NaN: {torch.isnan(xyz).any()}")
    print(f"  Valid residues per sample: {mask.sum(dim=1)}")
    
    return xyz, pair, idx, mask

def make_topk_graph_with_mask(xyz, pair, idx, mask, top_k=64, kmin=32, eps=1e-6):
    '''
    Input:
        - xyz: current backbone coordinates (B, L, 3, 3)
        - pair: pair features from Trunk (B, L, L, E)
        - idx: residue index from ground truth pdb
        - mask: mask indicating valid residues (B, L), 1=valid, 0=pad
    Output:
        - G: defined graph (excluding edges involving pad residues)
    '''

    B, L = xyz.shape[:2]
    device = xyz.device
    
    # Create mask matrix (B, L, L) where True indicates both residues are valid
    mask_mat = mask[:, :, None] * mask[:, None, :]  # (B, L, L)
    
    # Distance map from current CA coordinates
    D = torch.cdist(xyz, xyz) 
    # Add large value to diagonal and pad positions
    print(D.shape, mask_mat.shape)
    D = D + (
        torch.eye(L, device=device).unsqueeze(0)*999.9 + 
        (~mask_mat)*999.9
    )
    # Sequence separation
    sep = idx[:,None,:] - idx[:,:,None]
    sep = sep.abs() + torch.eye(L, device=device).unsqueeze(0)*999.9
    D = D + sep*eps
    
    # Get top_k neighbors (only valid pairs considered due to mask in D)
    D_neigh, E_idx = torch.topk(D, min(top_k, L), largest=False)  # (B, L, top_k)
    topk_matrix = torch.zeros((B, L, L), device=device)
    topk_matrix.scatter_(2, E_idx, 1.0)

    # Conditions for edges:
    #   1) |i-j| <= kmin (sequential neighbors)
    #   2) Top_k neighbors
    cond = torch.logical_or(topk_matrix > 0.0, sep < kmin)
    # Apply mask: only consider edges between valid residues
    cond = cond & mask_mat
    b, i, j = torch.where(cond)
    
    # Create graph
    src = b*L + i
    tgt = b*L + j
    G = dgl.graph((src, tgt), num_nodes=B*L).to(device)
    
    # Add relative position features (only for valid edges)
    G.edata['rel_pos'] = (xyz[b, j,:] - xyz[b, i,:]).detach()
    
    # Add pair features (only for valid edges)
    pair_features = pair[b, i, j].unsqueeze(-1)  # [E, E, 1]
    
    return G, pair_features

    
def test_se3_with_padded_batch():
    """
    测试SE3TransformerWrapper处理包含padding的batch数据
    """
    print("="*70)
    print("Testing SE3TransformerWrapper with Padded Batch Data")
    print("="*70)
    
    # 1. 创建包含padding的batch数据
    xyz, pair_features, idx, mask = create_batch_with_padding(real_lengths=[100,70,60])
    
    # 2. 使用修改的图构建函数
    print(f"\nBuilding graph with mask filtering...")
    graph, edge_features = make_topk_graph(
        xyz[:,:,1,:], pair_features, idx, top_k=32, kmin=9
    )
    print(f"  Graph created successfully!")
    print(f"  Nodes: {graph.num_nodes()}")
    print(f"  Edges: {graph.num_edges()}")
    print(f"  Edge features shape: {edge_features.shape}")
    print(f"  Graph has 'rel_pos': {'rel_pos' in graph.edata}")
    print(graph.edata['rel_pos'].shape)
    
    # 3. 准备SE3TransformerWrapper输入
    print(f"\nPreparing SE3 model inputs...")
    
    B, L = xyz.shape[:2]
    
    # 节点特征
    node_features = torch.randn(B*L, 32)
    
    # l1特征 (相对坐标)
    xyz_center = xyz[:,:,1,:]  # CA坐标
    l1_features = xyz - xyz_center.unsqueeze(2)  # 相对于CA的坐标
    l1_features = l1_features.reshape(B*L, -1, 3)
    
    # 将无效节点的特征设为0
    valid_nodes = mask.flatten()  # (B*L,)
    node_features[~valid_nodes] = 0
    l1_features[~valid_nodes] = 0
    
    print(f"  Node features shape: {node_features.shape}")
    print(f"  L1 features shape: {l1_features.shape}")
    print(f"  Edge features shape: {edge_features.shape}")
    print(f"  Valid nodes: {valid_nodes.sum()}/{len(valid_nodes)}")
    
    # 4. 初始化SE3TransformerWrapper
    print(f"\nInitializing SE3TransformerWrapper...")
    
    SE3_param = {
        'l0_in_features': 32,
        'l0_out_features': 16,
        'l1_in_features': 3,
        'l1_out_features': 2,
        'num_edge_features': 128,
        'num_layers': 2,
        'num_channels': 16,
        'num_degrees': 3,
        'n_heads': 4,
        'div': 4
    }

    se3_model = SE3TransformerWrapper(**SE3_param)
    se3_model.eval()
    print(f"  SE3 model initialized successfully!")
        
    
    # 5. 运行SE3模型
    print(f"\nRunning SE3TransformerWrapper...")
    
    try:
        with torch.no_grad():
            # 准备输入格式
            type_0_features = node_features.unsqueeze(-1)  # (B*L, 32, 1)
            type_1_features = l1_features  # (B*L, 9, 3)
            
            # 修正：确保边特征是2D的，因为SE3TransformerWrapper.forward会处理成字典格式
            if edge_features.dim() == 3:
                edge_features_input = edge_features.squeeze(-1)  # 移除最后一个维度 -> (num_edges, 128)
            else:
                edge_features_input = edge_features
            
            print(f"  Input shapes:")
            print(f"    Graph: {graph.num_nodes()} nodes, {graph.num_edges()} edges")
            print(f"    Type 0 features: {type_0_features.shape}")
            print(f"    Type 1 features: {type_1_features.shape}")
            print(f"    Edge features: {edge_features_input.shape}")
            print(f"    Rel pos shape: {graph.edata['rel_pos'].shape}")
            
            # 运行模型 - 传递2D边特征，让SE3TransformerWrapper内部处理
            output = se3_model(graph, type_0_features, type_1_features, edge_features_input)
            
            print(f"\n✅ SUCCESS! SE3TransformerWrapper processed padded batch data!")
            print(f"Output shapes:")
            for key, value in output.items():
                print(f"  {key}: {value.shape}")
            
            # 验证输出的有效性
            print(f"\nOutput validation:")
            for key, value in output.items():
                has_nan = torch.isnan(value).any()
                print(f"  {key} contains NaN: {has_nan}")
                if not has_nan:
                    print(f"  {key} range: [{value.min().item():.4f}, {value.max().item():.4f}]")
            
            return True
            
    except Exception as e:
        print(f"  ❌ SE3 model execution failed: {e}")
        import traceback
        traceback.print_exc()
        return False

if __name__ == "__main__":
    # 主测试
    success = test_se3_with_padded_batch()
    
    # 极端情况测试
    # test_extreme_cases()
    
    if success:
        print(f"\n🎉 All tests passed! SE3TransformerWrapper can handle padded batch data with the modified graph construction method.")
    else:
        print(f"\n⚠️  Some tests failed. Check the error messages above.")

Testing SE3TransformerWrapper with Padded Batch Data
Creating batch data with padding...
Batch size: 3
Max sequence length: 100
Real sequence lengths: [100, 70, 60]
Data shapes:
  XYZ coordinates: torch.Size([3, 100, 3, 3])
  Pair features: torch.Size([3, 100, 100, 128])
  Residue indices: torch.Size([3, 100])
  Valid mask: torch.Size([3, 100])
  Contains NaN: True
  Valid residues per sample: tensor([100,  70,  60])

Building graph with mask filtering...
  Graph created successfully!
  Nodes: 300
  Edges: 12159
  Edge features shape: torch.Size([12159, 128, 1])
  Graph has 'rel_pos': True
torch.Size([12159, 3])

Preparing SE3 model inputs...
  Node features shape: torch.Size([300, 32])
  L1 features shape: torch.Size([300, 3, 3])
  Edge features shape: torch.Size([12159, 128, 1])
  Valid nodes: 230/300

Initializing SE3TransformerWrapper...
  SE3 model initialized successfully!

Running SE3TransformerWrapper...
  Input shapes:
    Graph: 300 nodes, 12159 edges
    Type 0 features: tor

Traceback (most recent call last):
  File "/tmp/ipykernel_3611677/4171434672.py", line 197, in test_se3_with_padded_batch
    output = se3_model(graph, type_0_features, type_1_features, edge_features_input)
  File "/home/fit/lulei/WORK/.conda/envs/SE3nv/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/fit/lulei/WORK/.conda/envs/SE3nv/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1750, in _call_impl
    return forward_call(*args, **kwargs)
  File "/WORK/PUBLIC/lulei_work/xjt/Protein_design/RFdiffusion/rfdiffusion/SE3_network.py", line 83, in forward
    return self.se3(G, node_features, edge_features)
  File "/home/fit/lulei/WORK/.conda/envs/SE3nv/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/fit/lulei/WORK/.conda/envs/SE3nv/lib/python3.9/site-packages/torch/nn/modules/modu

In [61]:
def make_topk_graph(xyz, pair, idx, top_k=64, kmin=32, eps=1e-6):
    '''
    Input:
        - xyz: current backbone cooordinates (B, L, 3, 3)
        - pair: pair features from Trunk (B, L, L, E)
        - idx: residue index from ground truth pdb
    Output:
        - G: defined graph
    '''

    B, L = xyz.shape[:2]
    device = xyz.device
    
    # distance map from current CA coordinates
    print("aaa",torch.eye(L, device=device).unsqueeze(0).shape)
    print(torch.cdist(xyz[:,:,1,:], xyz[:,:,1,:]).shape )
    D = torch.cdist(xyz[:,:,1,:], xyz[:,:,1,:]) + torch.eye(L, device=device).unsqueeze(0)*999.9
    # seq sep
    sep = idx[:,None,:] - idx[:,:,None]
    sep = sep.abs() + torch.eye(L, device=device).unsqueeze(0)*999.9
    D = D + sep*eps
    
    # get top_k neighbors
    D_neigh, E_idx = torch.topk(D, min(top_k, L), largest=False) # shape of E_idx: (B, L, top_k)
    topk_matrix = torch.zeros((B, L, L), device=device)
    topk_matrix.scatter_(2, E_idx, 1.0)

    # put an edge if any of the 3 conditions are met:
    #   1) |i-j| <= kmin (connect sequentially adjacent residues)
    #   2) top_k neighbors
    cond = torch.logical_or(topk_matrix > 0.0, sep < kmin)
    b,i,j = torch.where(cond)
   
    src = b*L+i
    tgt = b*L+j
    G = dgl.graph((src, tgt), num_nodes=B*L).to(device)
    G.edata['rel_pos'] = (xyz[b,j,:] - xyz[b,i,:]).detach() # no gradient through basis function

    return G, pair[b,i,j][...,None]

xyz, pair_features, idx, mask = create_batch_with_padding(real_lengths=[100,100,100])

graph, edge_features = make_topk_graph(
    xyz, pair_features, idx, top_k=32, kmin=9
)
print(f"  Graph created successfully!")
print(f"  Nodes: {graph.num_nodes()}")
print(f"  Edges: {graph.num_edges()}")
print(f"  Edge features shape: {edge_features.shape}")
print(f"  Graph has 'rel_pos': {'rel_pos' in graph.edata}")
print(graph.edata['rel_pos'].shape)
    
# 4. 初始化SE3TransformerWrapper
print(f"\nInitializing SE3TransformerWrapper...")

SE3_param = {
    'l0_in_features': 32,
    'l0_out_features': 16,
    'l1_in_features': 3,
    'l1_out_features': 2,
    'num_edge_features': 128,
    'num_layers': 2,
    'num_channels': 16,
    'num_degrees': 3,
    'n_heads': 4,
    'div': 4
}


se3_model = SE3TransformerWrapper(**SE3_param)
se3_model.eval()
print(f"  SE3 model initialized successfully!")



# 5. 运行SE3模型
print(f"\nRunning SE3TransformerWrapper...")

with torch.no_grad():
    # 准备输入格式
    type_0_features = node_features.unsqueeze(-1)  # (B*L, 32, 1)
    type_1_features = l1_features  # (B*L, 9, 3)
    
    # 修正：确保边特征是2D的，因为SE3TransformerWrapper.forward会处理成字典格式
    if edge_features.dim() == 3:
        edge_features_input = edge_features.squeeze(-1)  # 移除最后一个维度 -> (num_edges, 128)
    else:
        edge_features_input = edge_features
    
    print(f"  Input shapes:")
    print(f"    Graph: {graph.num_nodes()} nodes, {graph.num_edges()} edges")
    print(f"    Type 0 features: {type_0_features.shape}")
    print(f"    Type 1 features: {type_1_features.shape}")
    print(f"    Edge features: {edge_features_input.shape}")
    print(f"    Rel pos shape: {graph.edata['rel_pos'].shape}")
    
    # 运行模型 - 传递2D边特征，让SE3TransformerWrapper内部处理
    output = se3_model(graph, type_0_features, type_1_features, edge_features_input)
    
    print(f"\n✅ SUCCESS! SE3TransformerWrapper processed padded batch data!")
    print(f"Output shapes:")


Creating batch data with padding...
Batch size: 3
Max sequence length: 100
Real sequence lengths: [100, 100, 100]
Data shapes:
  XYZ coordinates: torch.Size([3, 100, 3, 3])
  Pair features: torch.Size([3, 100, 100, 128])
  Residue indices: torch.Size([3, 100])
  Valid mask: torch.Size([3, 100])
  Contains NaN: False
  Valid residues per sample: tensor([100, 100, 100])
aaa torch.Size([1, 100, 100])
torch.Size([3, 100, 100])
  Graph created successfully!
  Nodes: 300
  Edges: 12699
  Edge features shape: torch.Size([12699, 128, 1])
  Graph has 'rel_pos': True
torch.Size([12699, 3, 3])

Initializing SE3TransformerWrapper...
  SE3 model initialized successfully!

Running SE3TransformerWrapper...


NameError: name 'node_features' is not defined

In [3]:
import torch
a = torch.tensor([[1,1,0,1]]).int()
b = a.unsqueeze(2) @ a.unsqueeze(1)
print(b)

tensor([[[1, 1, 0, 1],
         [1, 1, 0, 1],
         [0, 0, 0, 0],
         [1, 1, 0, 1]]], dtype=torch.int32)


In [1]:
from dataloader import *
data_dir = "/home/fit/lulei/WORK/xjt/Protein_design/CyclicPeptide/Dataset/ALL_PDB"
cluster_file = os.path.join(data_dir,"tmp/cluster.tsv")
pdb_feature_dir = os.path.join(data_dir,"PDB_FEATURE")
pdb_dir =  os.path.join(data_dir,"final_cleaned_PDB")
link_file_path = os.path.join(data_dir,"LINK/results.txt")
cache_dir = os.path.join(data_dir,"dataset_cache")
crop_size = 168
val_ratio=0.05
seed=42  # 随机种子分割验证集和训练集

# 第一部分：获取数据集
dataset_train = get_or_create_dataset(
    cluster_file=cluster_file,
    pdb_feature_dir=pdb_feature_dir,
    pdb_dir=pdb_dir,
    link_file_path=link_file_path,
    crop_size=crop_size,
    cache_dir=cache_dir,
    split='train', 
    val_ratio=val_ratio,
    seed=seed
)

dataset_val = get_or_create_dataset(
    cluster_file=cluster_file,
    pdb_feature_dir=pdb_feature_dir,
    pdb_dir=pdb_dir,
    link_file_path=link_file_path,
    crop_size=crop_size,
    cache_dir=cache_dir,
    split='val', 
    val_ratio=val_ratio,
    seed=seed
)


# 第二部分：创建数据加载器
dataloader = create_dataloader_from_dataset(
    dataset=dataset,
    device='cpu',
    batch_size=64,
    num_workers=8,
)


Creating new train dataset...
start construct Linkereader
1TON_A not equal
2P6Z_B not equal
验证 1PY3_B 时出错: string index out of range


Traceback (most recent call last):
  File "/WORK/PUBLIC/lulei_work/xjt/Protein_design/RFdiffusion/mytest/dataloader.py", line 151, in _verify_single_chain
    res2_name_pdb = AA_1_TO_3[pdb_residue_map[res2_num_link]]
IndexError: string index out of range


2BXU_H not equal
验证 1D7P_M 时出错: string index out of range


Traceback (most recent call last):
  File "/WORK/PUBLIC/lulei_work/xjt/Protein_design/RFdiffusion/mytest/dataloader.py", line 151, in _verify_single_chain
    res2_name_pdb = AA_1_TO_3[pdb_residue_map[res2_num_link]]
IndexError: string index out of range


1YC7_B not equal
1YC8_B not equal
1QDM_A not equal
1QDM_C not equal
验证 1GL3_A 时出错: string index out of range


...

Traceback (most recent call last):
  File "/WORK/PUBLIC/lulei_work/xjt/Protein_design/RFdiffusion/mytest/dataloader.py", line 151, in _verify_single_chain
    res2_name_pdb = AA_1_TO_3[pdb_residue_map[res2_num_link]]
IndexError: string index out of range


7OEF_B not equal
7OEJ_B not equal
验证 7RDK_B 时出错: string index out of range


Traceback (most recent call last):
  File "/WORK/PUBLIC/lulei_work/xjt/Protein_design/RFdiffusion/mytest/dataloader.py", line 151, in _verify_single_chain
    res2_name_pdb = AA_1_TO_3[pdb_residue_map[res2_num_link]]
IndexError: string index out of range


9G1Y_C not equal
9G2A_D not equal
8F7U_A not equal
8F7V_A not equal
9G6S_H not equal
9B75_H not equal
9HVN_A not equal
8DB4_H not equal
8HYG_A not equal
添加到黑名单: 1ATT_A (验证失败)
添加到黑名单: 1ATT_B (验证失败)
添加到黑名单: 1AVF_A (验证失败)
添加到黑名单: 1AVF_J (验证失败)
添加到黑名单: 1BNE_A (验证失败)
添加到黑名单: 1CHG_A (验证失败)
添加到黑名单: 1D7P_M (验证失败)
添加到黑名单: 1DZB_A (验证失败)
添加到黑名单: 1FDP_A (验证失败)
添加到黑名单: 1GL3_A (验证失败)
添加到黑名单: 1H3P_H (验证失败)
添加到黑名单: 1H8D_H (验证失败)
添加到黑名单: 1H8N_A (验证失败)
添加到黑名单: 1H8O_A (验证失败)
添加到黑名单: 1H8O_B (验证失败)
添加到黑名单: 1HGU_A (验证失败)
添加到黑名单: 1HL5_F (验证失败)
添加到黑名单: 1HRN_A (验证失败)
添加到黑名单: 1KCQ_A (验证失败)
添加到黑名单: 1N31_A (验证失败)
添加到黑名单: 1O7S_A (验证失败)
添加到黑名单: 1OEZ_W (验证失败)
添加到黑名单: 1PQU_A (验证失败)
添加到黑名单: 1PQU_C (验证失败)
添加到黑名单: 1PY3_B (验证失败)
添加到黑名单: 1QDM_A (验证失败)
添加到黑名单: 1QDM_C (验证失败)
添加到黑名单: 1QFW_H (验证失败)
添加到黑名单: 1QFW_L (验证失败)
添加到黑名单: 1QOK_A (验证失败)
添加到黑名单: 1RNE_A (验证失败)
添加到黑名单: 1RTF_B (验证失败)
添加到黑名单: 1TET_H (验证失败)
添加到黑名单: 1TON_A (验证失败)
添加到黑名单: 1UN4_A (验证失败)
添加到黑名单: 1V1Q_A (验证失败)
添加到黑名单: 1V1Q_B (验证失败)
添加到黑名单: 1YC7_B (验证失败)
添加到黑名单: 1YC

NameError: name 'dataset' is not defined